In [ ]:
%%bash
pip install pyaudio

In [ ]:
import pyaudio

# 查找设备索引
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
p.terminate()

In [ ]:
import pyaudio

# 查找设备索引
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    if dev['maxInputChannels'] == 0:
        continue
    print(f"设备索引：{i}, 设备名称：{dev['name']}, 输入通道数：{dev['maxInputChannels']}")
p.terminate()

In [ ]:
# ================================
# 录音配置（录制为 WAV 文件，后面转换为 MP3）
# ================================
import pyaudio
import wave

def record_audio(filename='output.wav', duration=30):
    chunk = 1024  # 每次读取的字节数
    format = pyaudio.paInt16  # 16-bit 音频
    channels = 2  # 立体声
    rate = 44100  # 采样率

    # 初始化 PyAudio
    audio = pyaudio.PyAudio()
    stream = audio.open(format=format, channels=channels, rate=rate, input=True, input_device_index=1, frames_per_buffer=chunk)

    print('录音中……')
    frames = []

    for i in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print('录音结束，保存文件……')
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # 保存 WAV 文件
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(audio.get_sample_size(format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

# ================================
# 使用 Selenium 加载 YouTube
# ================================
import time
import undetected_chromedriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

def load_youtube_video(video_url):
    options = Options()
    # options.add_argument('--headless')  # 无界面模式
    # options.add_argument('--mute-audio')  # 静音浏览器

    driver = undetected_chromedriver.Chrome(options=options)
    driver.maximize_window()
    driver.get(video_url)

    # 等待 YouTube 视频加载（确保已经进入播放器区域）
    time.sleep(3)
    print('YouTube 视频加载完成！')

    return driver

# ================================
# 使用 ffmpeg 将 WAV 转换为 MP3
# ================================
import ffmpeg

def convert_wav_to_mp3(input_file='output.wav', output_file='output.mp3'):
    print(f'转换 {input_file} 到 {output_file} 中……')
    try:
        ffmpeg.input(input_file).output(output_file).run(cmd='./ffmpeg-7.1.1-essentials_build/bin/ffmpeg.exe')
        print('转换完成！')
    except Exception as e:
        print(f'转换时出错：{e}')

# ================================
# 主函数
# ================================
import threading

if __name__ == '__main__':
    youtube_url = 'https://www.youtube.com/watch?v=XR4u4jQ4bmo'  # 替换为目标视频 URL
    recording_time = 60 * 5  # 录音时长（秒）

    # 加载 YouTube 视频
    driver = load_youtube_video(youtube_url)

    # 创建录音线程
    audio_thread = threading.Thread(target=record_audio, args=('output.wav', recording_time))
    audio_thread.start()

    # 自动播放视频
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.SPACE)  # 模拟按下空格键播放视频
    print('YouTube 视频开始播放……')

    # 等待录音完成
    audio_thread.join()
    # 关闭浏览器
    driver.quit()

    # 转换录音为 MP3 格式
    convert_wav_to_mp3('output.wav', 'output.mp3')
    print('YouTube 音频保存完成！')

In [ ]:
import ffmpeg

def trim_mp3(input_file, output_file, start_time=0, duration=270):
    """
    截取 MP3 文件的前指定时长部分（默认 4 分 30 秒，270 秒）

    参数:
    input_file (str): 输入 MP3 文件路径
    output_file (str): 输出截取后 MP3 文件路径
    start_time (int): 起始时间（秒）
    duration (int): 截取时长（秒）
    """
    try:
        print(f"正在从 {input_file} 中截取前 {duration} 秒...")
        # FFmpeg 截取音频
        ffmpeg.input(input_file, ss=start_time, t=duration).output(output_file).run(overwrite_output=True, cmd='./ffmpeg-7.1.1-essentials_build/bin/ffmpeg.exe')
        print(f"截取完成！输出文件：{output_file}")
    except Exception as e:
        print(f"截取音频时出错：{e}")

# 调用示例：截取前 4 分 30 秒
trim_mp3("202503211930.mp3", "output_trimmed.mp3")

正在从 202503211930.mp3 中截取前 270 秒...
截取完成！输出文件：output_trimmed.mp3


In [1]:
import ffmpeg

def repeat_mp3(input_file, output_file, repeat_count=100):
    """
    将 MP3 文件重复多次并保存为新文件

    参数:
    input_file (str): 输入 MP3 文件路径
    output_file (str): 输出重复后的 MP3 文件路径
    repeat_count (int): 重复次数，默认为 100
    """
    try:
        print(f"正在将 {input_file} 重复 {repeat_count} 次...")

        # 使用 FFmpeg 过滤器重复音频
        ffmpeg.input(input_file, stream_loop=repeat_count - 1).output(output_file).run(overwrite_output=True, cmd='./ffmpeg-7.1.1-essentials_build/bin/ffmpeg.exe')

        print(f"重复完成！输出文件：{output_file}")
    except Exception as e:
        print(f"音频重复时出错：{e}")

# 调用示例：重复 100 遍并保存为 output_repeated.mp3
repeat_mp3("output_trimmed.mp3", "output_repeated.mp3")

正在将 output_trimmed.mp3 重复 100 次...
重复完成！输出文件：output_repeated.mp3


In [ ]:
%%bash
pip install spleeter

In [ ]:
import os

# 手动指定 ffmpeg 路径
os.environ['PATH'] += os.pathsep + os.path.abspath('./ffmpeg-7.1.1-essentials_build/bin')

from spleeter.separator import Separator

# 初始化 Spleeter 使用2stem模型（vocal + accompaniment）
separator = Separator('spleeter:2stems')

# 输入文件路径
input_audio = 'output_trimmed.mp3'

# 输出到当前目录的 'separated_audio' 文件夹
separator.separate_to_file(input_audio, 'separated_audio')